In [ ]:
from keras.models import Model
from keras import layers
from keras import Input
import keras
import numpy as np

dim=978

#训练数据
train_data=np.load(r'after_merge/A549/train_data.npy')
train_target=np.load(r'after_merge/A549/train_target.npy')

In [ ]:
input1=train_data.reshape(train_data.shape[0],train_data.shape[1]*train_data.shape[2])
input2=train_data
output=train_target

In [ ]:
wide_input=Input(shape=(input1.shape[1],),name='wide')
wide_output=layers.Dense(1)(wide_input)

deep_input=Input(shape=(input2.shape[1],input2.shape[2],),name='deep')
deep_hide=layers.LSTM(400，return_sequences=True)(deep_input)
deep_hide=layers.LSTM(400)(deep_hide)
deep_output=layers.Dense(1)(deep_hide)

concatenated = layers.concatenate([wide_output,deep_output],axis = -1)

final_output=layers.Dense(1,activation = 'sigmoid')(concatenated)

model = Model([wide_input,deep_input],final_output)
#rmsprop
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['acc'])

#model.summary()

model.fit([input1,input2],output,epochs=20,batch_size=64)

In [ ]:
test_data=np.load(r'after_merge/A549/test_data.npy')
test_target=np.load(r'after_merge/A549/test_target.npy')
test_input1=test_data.reshape(test_data.shape[0],test_data.shape[1]*test_data.shape[2])
test_input2=test_data

In [ ]:
model.evaluate([test_input1,test_input2], test_target,verbose=1,batch_size=1)

In [ ]:
pred=model.predict([test_input1,test_input2])
for pred_i in pred:
    pred_i[pred_i >=0.5] = 1
    pred_i[pred_i < 0.5] = 0
from sklearn.metrics import accuracy_score ,f1_score
accuracy_score(test_target, pred.flatten(),normalize =0)